This one is on hold. Need an experimental build of Opencv (3.4.11) before it can be run.

In [1]:
import glob as glob
import os

import cv2
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.metrics import (
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    roc_auc_score,
)

from yolov4.tf import YOLOv4

In [2]:
yolo = YOLOv4()
yolo.classes = "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/coco.names"

yolo.make_model()
yolo.load_weights(
    "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov4.weights",
    weights_type="yolo",
)

In [3]:
test_image = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/YOLO_Training/Test Data/1106vid2image00544.jpg"

In [9]:
x = yolo.inference(test_image)

time: 4405.44 ms
YOLOv4: Inference is finished


In [10]:
x

In [2]:
# Load the labels
LABELS = open("/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/coco.names").read().strip().split("\n")

In [3]:
# Get the weights and config
configpath = "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov4.cfg"
weightspath = "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov4.weights"

net = cv2.dnn.readNetFromDarknet(configpath, weightspath)

# Determine the output layer names
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

error: OpenCV(4.3.0) ../modules/dnn/src/darknet/darknet_io.cpp:821: error: (-212:Parsing error) Unsupported activation: mish in function 'ReadDarknetFromCfgStream'


In [119]:
# Function to make a prediction and save to df
def predict_and_save(folder, coded_data, conf_thresh=0.25, nms_thresh=0.2):
    predict_files = glob.glob(folder + "/*/*.jpg")

    predictor, prob, image_id = [], [], []
    final = pd.DataFrame(columns=["id", "prediction", "confidence", "image"])

    not_coded = []

    for index, image in enumerate(predict_files):
        if coded_data["filename"].str.contains(os.path.basename(image)).any():
            clear_output(wait=True)
            print(f"Working on image {index} of {len(predict_files)-1}")

            im = cv2.imread(image)
            (H, W) = im.shape[:2]

            # Create the blob
            blob = cv2.dnn.blobFromImage(
                im, 1 / 255.0, (416, 416), swapRB=True, crop=False
            )
            net.setInput(blob)
            layerOutputs = net.forward(ln)

            # Translate the predictions
            boxes = []
            confidences = []
            classIDs = []

            for output in layerOutputs:
                for detection in output:
                    scores = detection[5:]
                    classID = np.argmax(scores)
                    confidence = scores[classID]

                    if confidence > conf_thresh:
                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")
                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))
                        # update our list of bounding box coordinates, confidences, and class IDs
                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        classIDs.append(classID)

            # apply non-maxima suppression to suppress weak, overlapping bounding boxes
            idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf_thresh, nms_thresh)

            # Append to df
            if len(idxs):
                for i in idxs.flatten():
                    final = final.append(
                        {
                            "id": classIDs[i],
                            "prediction": LABELS[classIDs[i]],
                            "confidence": confidences[i],
                            "image": os.path.basename(image),
                        },
                        ignore_index=True,
                    )
            else:  # no predictions made
                final = final.append(
                    {
                        "id": None,
                        "prediction": None,
                        "confidence": None,
                        "image": os.path.basename(image),
                    },
                    ignore_index=True,
                )

    return final

##  Test against Bridget's coding

In [120]:
%%time
coded_data = pd.read_csv("/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images/Screen Time Coding Data - Device.csv")
folder = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images"
df = predict_and_save(folder, coded_data)

Working on image 4495 of 4495
CPU times: user 2h 4min 30s, sys: 1min 46s, total: 2h 6min 17s
Wall time: 34min 21s


In [121]:
df_backup = df.copy()

In [122]:
cat_maybe = [
    "mouse",
    "remote",
    "keyboard",
]

cat_def = [
    "tvmonitor",
    "laptop",
    "cell phone",
]

In [123]:
df['screen_def'] = np.where(df["prediction"].isin(cat_def),1,0)
df['screen_maybe'] = np.where(df["prediction"].isin(cat_def + cat_maybe),1,0)

In [124]:
def conf_def(df, confthresh):
    if df["confidence"] > confthresh and df["screen_def"]==1:
        return 1
    else:
        return 0
def conf_maybe(df, confthresh):
    if df["confidence"] > confthresh and df["screen_maybe"]==1:
        return 1
    else:
        return 0

In [125]:
for x in np.arange(0.3, 0.8,0.05):
    df["screen_def_"+str(x)] = df.apply(conf_def, confthresh=x, axis=1)
    df["screen_maybe_"+str(x)] = df.apply(conf_maybe, confthresh=x, axis=1)

In [126]:
df = df.merge(coded_data,left_on="image", right_on="filename")
df.drop(columns="filename", inplace=True)
df[["device","device_excl_bkg"]] = df[["device","device_excl_bkg"]].astype(int)
df = df.drop(columns=["prediction","confidence","id"]).groupby(["image"]).any()

In [127]:
df.columns

Index(['screen_def', 'screen_maybe', 'screen_def_0.3', 'screen_maybe_0.3',
       'screen_def_0.35', 'screen_maybe_0.35',
       'screen_def_0.39999999999999997', 'screen_maybe_0.39999999999999997',
       'screen_def_0.44999999999999996', 'screen_maybe_0.44999999999999996',
       'screen_def_0.49999999999999994', 'screen_maybe_0.49999999999999994',
       'screen_def_0.5499999999999999', 'screen_maybe_0.5499999999999999',
       'screen_def_0.5999999999999999', 'screen_maybe_0.5999999999999999',
       'screen_def_0.6499999999999999', 'screen_maybe_0.6499999999999999',
       'screen_def_0.7', 'screen_maybe_0.7', 'screen_def_0.7499999999999999',
       'screen_maybe_0.7499999999999999', 'device', 'device_excl_bkg'],
      dtype='object')

In [129]:
true_devices = ['device', 'device_excl_bkg']
predicted_devices = df.drop(columns=['device', 'device_excl_bkg','screen_def', 'screen_maybe',])


In [130]:
for true_device in true_devices:
    for predicted_device in predicted_devices:
        print(f"Comparing: {true_device} & {predicted_device}")
        print(classification_report(df[true_device], df[predicted_device]))

Comparing: device & screen_def_0.3
              precision    recall  f1-score   support

       False       0.60      0.96      0.74      1484
        True       0.97      0.68      0.80      3011

    accuracy                           0.77      4495
   macro avg       0.78      0.82      0.77      4495
weighted avg       0.85      0.77      0.78      4495

Comparing: device & screen_maybe_0.3
              precision    recall  f1-score   support

       False       0.64      0.96      0.77      1484
        True       0.97      0.73      0.84      3011

    accuracy                           0.81      4495
   macro avg       0.81      0.85      0.80      4495
weighted avg       0.86      0.81      0.81      4495

Comparing: device & screen_def_0.35
              precision    recall  f1-score   support

       False       0.58      0.96      0.73      1484
        True       0.97      0.66      0.79      3011

    accuracy                           0.76      4495
   macro avg       0

In [134]:
print(f"Comparing: device & screen_maybe_0.3")
print(classification_report(df["device"], df["screen_maybe_0.3"]))
print(cohen_kappa_score(df["device"], df["screen_maybe_0.3"]))
print(f"Comparing: device_excl_bkg & screen_maybe_0.3")
print(classification_report(df["device_excl_bkg"], df["screen_maybe_0.3"]))
print(cohen_kappa_score(df["device_excl_bkg"], df["screen_maybe_0.3"]))

Comparing: device & screen_maybe_0.3
              precision    recall  f1-score   support

       False       0.64      0.96      0.77      1484
        True       0.97      0.73      0.84      3011

    accuracy                           0.81      4495
   macro avg       0.81      0.85      0.80      4495
weighted avg       0.86      0.81      0.81      4495

0.6157554480285313
Comparing: device_excl_bkg & screen_maybe_0.3
              precision    recall  f1-score   support

       False       0.66      0.93      0.77      1573
        True       0.95      0.74      0.83      2922

    accuracy                           0.81      4495
   macro avg       0.80      0.83      0.80      4495
weighted avg       0.85      0.81      0.81      4495

0.6092436327761945


In [133]:

print(cohen_kappa_score(df["device_excl_bkg"], df["screen_maybe_0.3"]))

0.6092436327761945


In [140]:
print(roc_auc_score(df["device"], df["screen_maybe_0.3"]))

0.8465963972173907
